# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd
import numpy
import sys
import mlxtend
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth, fpmax

In [2]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [3]:
countries_data = data.query("country in ('Belarus', 'Ukraine', 'Poland', 'Russian Federation')")
countries_data.head(5)

,user,artist,sex,country
490,35,radiohead,m,Ukraine
491,35,the kooks,m,Ukraine
492,35,coldplay,m,Ukraine
493,35,nine inch nails,m,Ukraine
494,35,hot chip,m,Ukraine


In [4]:
fr=countries_data.groupby(['user','sex','country'])['artist'].apply(lambda x: '|'.join(x)).reset_index()
fr

,user,sex,country,artist
0,35,m,Ukraine,radiohead|the kooks|coldplay|nine inch nails|h...
1,39,m,Russian Federation,britney spears|a-ha|joss stone|christina aguil...
2,50,f,Russian Federation,the ataris|yann tiersen|the smashing pumpkins|...
3,64,m,Poland,apparat|drowning pool|armin van buuren|alexiso...
4,82,m,Russian Federation,p.o.d.|chimaira|arch enemy|scar symmetry|slipk...
...,...,...,...,...
2002,19703,f,Poland,the offspring|pearl jam|the smashing pumpkins|...
2003,19706,f,Russian Federation,enigma|pink floyd|the offspring|scorpions
2004,19710,f,Russian Federation,vnv nation|nine inch nails|kmfdm|apoptygma ber...
2005,19712,m,Poland,john williams|abba|vangelis|hans zimmer|céline...


In [5]:

prep_results = pd.concat([fr[['user']], fr['artist'].str.get_dummies('|')], axis = 1)
prep_results.head(10)

,user,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,...,weezer,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,zero 7,Édith piaf
0,35,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,50,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
3,64,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,82,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,83,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,84,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,92,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,113,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,129,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
apr_res = apriori(prep_results.drop(columns = ['user']), min_support=0.01, use_colnames=True)

apr_res.sort_values(by = ['support'], ascending=False)

,support,itemsets
309,0.129048,(metallica)
478,0.123568,(the prodigy)
393,0.122073,(red hot chili peppers)
386,0.114599,(radiohead)
102,0.112108,(coldplay)
...,...,...
726,0.010463,"(coma, linkin park)"
724,0.010463,"(coma, koЯn)"
1187,0.010463,"(portishead, red hot chili peppers)"
721,0.010463,"(coma, disturbed)"


In [7]:
rules = association_rules(apr_res, metric="confidence", min_threshold=0.5)
rules.sort_values(by = ['support'], ascending=False)

rules[ (rules['antecedents'].apply(lambda x: len(x)) >= 2) & (rules['confidence'] > 0.5) & (rules['lift'] > 1) & (rules['conviction'] > 2)].sort_values(by = ['support'], ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
37,"(akurat, pidżama porno)",(happysad),0.020927,0.053812,0.014449,0.690476,12.831349,0.013323,3.056916
38,"(akurat, happysad)",(pidżama porno),0.023916,0.050324,0.014449,0.604167,12.005569,0.013246,2.399182
39,"(pidżama porno, happysad)",(akurat),0.022920,0.048331,0.014449,0.630435,13.044151,0.013342,2.575105
59,"(koЯn, linkin park)",(system of a down),0.019930,0.106129,0.012955,0.650000,6.124648,0.010839,2.553918
51,"(massive attack, björk)",(portishead),0.018934,0.074738,0.011958,0.631579,8.450526,0.010543,2.511424
50,"(portishead, björk)",(massive attack),0.021923,0.080219,0.011958,0.545455,6.799548,0.010199,2.023518
62,"(led zeppelin, the doors)",(pink floyd),0.017937,0.099153,0.011460,0.638889,6.443467,0.009681,2.494653
35,"(akurat, myslovitz)",(happysad),0.018435,0.053812,0.011460,0.621622,11.551802,0.010468,2.500641
36,"(happysad, myslovitz)",(akurat),0.020927,0.048331,0.011460,0.547619,11.330633,0.010448,2.103690
44,"(arctic monkeys, coldplay)",(franz ferdinand),0.019930,0.056303,0.010962,0.550000,9.768584,0.009840,2.097105


In [8]:
fp_res = fpgrowth(prep_results.drop(columns = ['user']), min_support=0.01, use_colnames=True)
fp_res.sort_values(by = ['support'], ascending=False)

,support,itemsets
155,0.129048,(metallica)
108,0.123568,(the prodigy)
58,0.122073,(red hot chili peppers)
0,0.114599,(radiohead)
1,0.112108,(coldplay)
...,...,...
565,0.010463,"(portishead, red hot chili peppers)"
842,0.010463,"(rammstein, pink floyd)"
561,0.010463,"(portishead, nirvana)"
553,0.010463,"(massive attack, metallica)"


In [21]:
rules = association_rules(apr_res, metric="confidence", min_threshold=0.5)
rules.sort_values(by = ['support'], ascending=False)

rules[ (rules['antecedents'].apply(lambda x: len(x)) >=2) & (rules['confidence'] > 0.5) & (rules['lift'] > 5) & (rules['conviction'] > 1.2)].sort_values(by = ['support'], ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
37,"(akurat, pidżama porno)",(happysad),0.020927,0.053812,0.014449,0.690476,12.831349,0.013323,3.056916
38,"(akurat, happysad)",(pidżama porno),0.023916,0.050324,0.014449,0.604167,12.005569,0.013246,2.399182
39,"(pidżama porno, happysad)",(akurat),0.022920,0.048331,0.014449,0.630435,13.044151,0.013342,2.575105
59,"(koЯn, linkin park)",(system of a down),0.019930,0.106129,0.012955,0.650000,6.124648,0.010839,2.553918
67,"(placebo, nirvana)",(muse),0.022920,0.101644,0.012456,0.543478,5.346867,0.010127,1.967827
50,"(portishead, björk)",(massive attack),0.021923,0.080219,0.011958,0.545455,6.799548,0.010199,2.023518
70,"(the killers, muse)",(placebo),0.022422,0.102641,0.011958,0.533333,5.196117,0.009657,1.922913
51,"(massive attack, björk)",(portishead),0.018934,0.074738,0.011958,0.631579,8.450526,0.010543,2.511424
35,"(akurat, myslovitz)",(happysad),0.018435,0.053812,0.011460,0.621622,11.551802,0.010468,2.500641
36,"(happysad, myslovitz)",(akurat),0.020927,0.048331,0.011460,0.547619,11.330633,0.010448,2.103690


In [22]:
max_res=fpmax(prep_results.drop(columns = ['user']), min_support=0.01, use_colnames=True)
max_res.sort_values(by = ['support'], ascending=False)

,support,itemsets
713,0.051320,([unknown])
538,0.037867,(tori amos)
537,0.037867,(clint mansell)
500,0.035874,(god is an astronaut)
501,0.035874,(boards of canada)
...,...,...
420,0.010463,"(bonobo, the cinematic orchestra)"
867,0.010463,"(u2, massive attack)"
878,0.010463,"(coldplay, u2, radiohead)"
879,0.010463,"(30 seconds to mars, slipknot)"


In [24]:
rules = association_rules(apr_res, metric="confidence", min_threshold=0.5)
rules.sort_values(by = ['support'], ascending=False)

rules[ (rules['antecedents'].apply(lambda x: len(x)) >=2) & (rules['confidence'] > 0.75) & (rules['lift'] > 5) & (rules['conviction'] > 4)].sort_values(by = ['support'], ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
32,"(armin van buuren, above & beyond)",(paul van dyk),0.014449,0.030892,0.010962,0.758621,24.557286,0.010515,4.014877
33,"(paul van dyk, above & beyond)",(armin van buuren),0.012955,0.038864,0.010962,0.846154,21.772189,0.010458,6.247384
48,"(atb, ferry corsten)",(armin van buuren),0.011958,0.038864,0.010463,0.875000,22.514423,0.009999,7.689088
